In [1]:
from jupyter_dash import JupyterDash

In [3]:
JupyterDash.infer_jupyter_proxy_config()

In [4]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


### data
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,steward,count(tree_id)' +\
        '&$group=spc_common,boroname,health,steward' +\
        '&$limit=50000').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

df1 = soql_trees.groupby(['boroname','health'])['count_tree_id'].sum().groupby(level=0).apply(lambda x: round(100*x/x.sum(), 2)).reset_index().rename(columns={'boroname':'borough','count_tree_id':'pct'})

###

fig = px.bar(df1, x="borough", y="pct", color="health", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Proportion of Health - Borough'),

    html.Div(children='''
        Proportion of tree health by borough.
    ''', style={
        'textAlign': 'center'
    }),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


In [6]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,steward,count(tree_id)' +\
        '&$group=spc_common,boroname,health,steward' +\
        '&$limit=50000').replace(' ', '%20')
df = pd.read_json(soql_url)

df2 = soql_trees.groupby(['spc_common','health','steward'])['count_tree_id'].sum().groupby(level=0).apply(lambda x: round(100*x/x.sum(), 2)).reset_index().rename(columns={'spc_common':'tree','count_tree_id':'pct'})
df2['tree'] = df2['tree'].str.title()
df3 = soql_trees.groupby(['health','steward'])['count_tree_id'].sum().groupby(level=0).apply(lambda x: round(100*x/x.sum(), 2)).reset_index().rename(columns={'count_tree_id':'pct'})

fig1 = px.bar(df2, x="health", y="pct", color="steward", barmode="group", title="")
fig2 = px.bar(df3, x="health", y="pct", color="steward", barmode="group", title="Health by # of Stewards - Overall")


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                options=[x for x in df2.tree.unique()],
                id='tree-selector',
                value=[x.title() for x in df2.tree.unique()][0]
            )
        ],
        style={'width': '49%', 'display': 'inline-block'})
    ]),

    html.Div([
        dcc.Graph(
            id='tree-bar',
            figure=fig1
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}), 
    
    html.Div([
        dcc.Graph(
            id='health-bar',
            figure=fig2
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
])

@app.callback(
    Output('tree-bar','figure'),
    Input('tree-selector','value'))

def update_graph(value):
    df4 = df2[df2['tree'] == value]

    fig1 = px.bar(df4, x="health", 
                  y="pct", color="steward", 
                  barmode="group", 
                  title=value)

    return fig1


if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
